In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score,precision_score

from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
from collections import Counter

In [152]:
train = pd.read_csv('Train_Set_90621.csv')
train = train.drop(['Application ID','Bank Masked','Name Masked','Approved_Timestamp','Term','Male to Female Employees Ratio'],axis=1)


In [153]:
train['Approved_Year'] = train['Approved_Year'].astype(object)


train['New Business'] = train['New Business'].replace(to_replace=2.0,value=0.0)
train['New Business'] = train['New Business'].astype(object)

train['Interest Rate'] = train['Interest Rate'].astype(object)


train['Term_years'] = train['Term_years'].replace(to_replace=1900,value=2011)


train['City or Rural'] = train['City or Rural'].replace(to_replace=2,value=0)
train['City or Rural'] = train['City or Rural'].astype(object)


train['New Business']=train['New Business'].fillna(train['New Business'].mode()[0])

train['Bank Type'].fillna(train['Bank Type'].mode()[0],inplace=True)
train['Business Owner State'].fillna(train['Business Owner State'].mode()[0],inplace=True)
train['BankState'].fillna(train['BankState'].mode()[0],inplace=True)
train['Carry-forward Credit'].fillna(train['Carry-forward Credit'].mode()[0],inplace=True)
train['Documents Provided'].fillna(train['Documents Provided'].mode()[0],inplace=True)

train['Expected Company Income'].fillna(np.mean(train['Expected Company Income']),inplace=True)

In [154]:
train_num = train.select_dtypes(include = [np.number])
print(train_num.columns)
train_cat = train.select_dtypes(include = [np.object])
print(train_cat.columns)

Index(['Business_Industry_Type_Code', 'New Business', 'Employees',
       'Gross Disbursed Amount', 'Term_years', 'Jobs Retained',
       'Expected Company Income', 'Funds available with company',
       'Gross_Apprv_Amount', 'Company Branch Code', 'Jobs Generated',
       'Balance Left', 'Amount Defaulted', 'Final_Appved_Amount',
       'Default_Status'],
      dtype='object')
Index(['Bank Type', 'Business Owner State', 'Approved_Year', 'BankState',
       'Interest Rate', 'City or Rural', 'Carry-forward Credit',
       'Documents Provided'],
      dtype='object')


In [155]:
train_num.drop(['Default_Status'],axis=1,inplace=True)

C:\Users\aniru\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [142]:
X_norm = MinMaxScaler()

num_norm = X_norm.fit_transform(train_num)

X = pd.DataFrame(num_norm, columns = train_num.columns)

X.head()

,Business_Industry_Type_Code,New Business,Employees,Gross Disbursed Amount,Term_years,Jobs Retained,Expected Company Income,Funds available with company,Gross_Apprv_Amount,Company Branch Code,Jobs Generated,Balance Left,Amount Defaulted,Final_Appved_Amount
0,0.564507,1.0,0.00189,0.025323,0.473684,0.002045,0.000081,0.283579,0.052962,0.278804,0.000341,0.0,0.000000,0.044145
1,0.000001,1.0,0.00090,0.004157,0.157895,0.001136,0.000039,0.723182,0.006960,0.000000,0.000000,0.0,0.016925,0.003867
2,0.346088,0.0,0.00072,0.002389,0.184211,0.001136,0.000031,0.947222,0.004960,0.000000,0.000227,0.0,0.000000,0.004700
3,0.000001,1.0,0.00018,0.053034,0.657895,0.000227,0.000008,0.936161,0.110964,0.000011,0.000000,0.0,0.000000,0.092480
4,0.000001,0.0,0.00018,0.003822,0.342105,0.000000,0.000008,0.951248,0.007960,0.000011,0.000000,0.0,0.000000,0.007534


In [156]:
train_cat = pd.get_dummies(train_cat,drop_first=True)
train_cat.shape

(195118, 52)

In [157]:
X = pd.concat([train_num,train_cat],axis=1)

In [158]:
X.shape

(195118, 66)

In [159]:
y = train['Default_Status']

In [53]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
vf=[vif(X_vif.values,i) for i in range(X_vif.shape[1])]
vif_df = pd.DataFrame(vf,index=X_vif.columns,columns=['vif'])

In [54]:
vif_df.sort_values(by='vif',ascending=False)

,vif
Final_Appved_Amount,10.014972
Gross Disbursed Amount,9.246669
Approved_Year_2011,6.787643
Business Owner State_Karnataka,5.448307
BankState_Karnataka,5.437317
...,...
Company Branch Code,1.027549
Funds available with company,1.026674
Jobs Retained,1.002551
BankState_Goa,1.000644


In [161]:
X_vif= X.drop(['Jobs Generated','Gross_Apprv_Amount','BankState_Delhi'],axis=1)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X_vif, y, test_size=0.3, random_state=42)

from imblearn.over_sampling import SMOTE
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
from collections import Counter

counter = Counter(y_train)
print('Before',counter)
# oversampling the train dataset using SMOTE
smt = SMOTE()
#X_train, y_train = smt.fit_resample(X_train, y_train)
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

counter = Counter(y_train_sm)
print('After',counter)

Before Counter({0: 111810, 1: 24772})
After Counter({0: 111810, 1: 111810})


In [135]:
X_train, X_test, y_train, y_test = train_test_split(X_vif, y, test_size=0.3, random_state=42)
tuned_paramaters = [{'criterion': ['entropy', 'gini'],
                     'n_estimators': [100],
                     'max_depth': [15, 20],
                     'min_samples_leaf': [1, 5, 9],
                     'max_leaf_nodes': [2, 5]}]
# instantiate the 'DecisionTreeClassifier' 
# pass the 'random_state' to obtain the same samples for each time you run the code
decision_tree_classification = RandomForestClassifier(random_state = 10)

# use GridSearchCV() to find the optimal value of the hyperparameters
# estimator: pass the decision tree classifier model
# param_grid: pass the list 'tuned_parameters'
# cv: number of folds in k-fold i.e. here cv = 5
tree_grid = GridSearchCV(estimator = decision_tree_classification, 
                         param_grid = tuned_paramaters, 
                         cv = 5)

# fit the model on X_train and y_train using fit()
tree_grid_model = tree_grid.fit(X_train, y_train)

# get the best parameters
print('Best parameters for decision tree classifier: ', tree_grid_model.best_params_, '\n')

Best parameters for decision tree classifier:  {'criterion': 'gini', 'max_depth': 15, 'max_leaf_nodes': 5, 'min_samples_leaf': 1, 'n_estimators': 100} 



In [163]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
rfc = RandomForestClassifier(n_estimators=100,criterion='gini',max_depth= 15, max_leaf_nodes=5)
rfc_model = rfc.fit(X_train,y_train)
y_pred = rfc_model.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.835417520841875
[[48027    43]
 [ 9591   875]]
              precision    recall  f1-score   support

           0       0.83      1.00      0.91     48070
           1       0.95      0.08      0.15     10466

    accuracy                           0.84     58536
   macro avg       0.89      0.54      0.53     58536
weighted avg       0.85      0.84      0.77     58536



In [87]:
test = pd.read_csv('Test_Set_90621.csv')
test = test.drop(['Application ID','Bank Masked','Name Masked','Approved_Timestamp','Term','Male to Female Employees Ratio'],axis=1)


In [88]:
test['Approved_Year'] = test['Approved_Year'].astype(object)


test['New Business'] = test['New Business'].replace(to_replace=2.0,value=0.0)
test['New Business'] = test['New Business'].astype(object)


test['Interest Rate'] = test['Interest Rate'].replace(to_replace=1,value=2)
test['Interest Rate'] = test['Interest Rate'].astype(object)


test['Term_years'] = train['Term_years'].replace(to_replace=1900,value=2011)


test['City or Rural'] = test['City or Rural'].replace(to_replace=2,value=0)
test['City or Rural'] = test['City or Rural'].astype(object)


test['New Business']=test['New Business'].fillna(test['New Business'].mode()[0])

test['Bank Type'].fillna(test['Bank Type'].mode()[0],inplace=True)
test['Business Owner State'].fillna(test['Business Owner State'].mode()[0],inplace=True)
test['BankState'].fillna(test['BankState'].mode()[0],inplace=True)
test['Carry-forward Credit'].fillna(test['Carry-forward Credit'].mode()[0],inplace=True)
test['Documents Provided'].fillna(test['Documents Provided'].mode()[0],inplace=True)

test['Expected Company Income'].fillna(np.mean(test['Expected Company Income']),inplace=True)

In [89]:
test_df = pd.get_dummies(test,drop_first=True)
test_df.shape

(83623, 66)

In [90]:
test_df = test_df.drop(['Jobs Generated','Gross_Apprv_Amount','BankState_Delhi'],axis=1)

In [136]:
test = pd.read_csv('Test_Set_90621.csv')
y_pred = tree_grid_model.predict(test_df)
y_pred_series = pd.Series(y_pred)

app = test['Application ID']

data = {"Application ID": app,
        "Default_Status": y_pred_series}
  
# Concatenating the series side
# by side as depicted by axis=1
# If you want to concatenate the 
# series one below the other
# change the axis to zero.
df_samp_rf = pd.concat(data,
               axis = 1)
  
# show the dataframe
df_samp_rf.Default_Status.value_counts()

0    83366
1      257
Name: Default_Status, dtype: int64

In [92]:
df_samp_rf.set_index("Application ID", inplace = True)
submission_data_eda = df_samp_rf.to_csv('file_rf_with_sscale_smote_vif.csv', index = True)

In [94]:
X_vif.head()

,Business_Industry_Type_Code,New Business,Employees,Gross Disbursed Amount,Term_years,Jobs Retained,Expected Company Income,Funds available with company,Company Branch Code,Balance Left,...,Interest Rate_6,Interest Rate_7,Interest Rate_8,Interest Rate_9,Interest Rate_10,Interest Rate_11,Interest Rate_12,City or Rural_1,Carry-forward Credit_Y,Documents Provided_Y
0,0.477820,0.625877,0.175933,0.217007,1.332567,0.026460,-0.084280,-0.732054,1.770622,-0.002318,...,0,0,0,0,0,0,0,1,0,0
1,-1.508153,0.625877,0.006745,-0.546814,-0.494006,-0.005702,-0.094702,0.902245,-0.218076,-0.002318,...,0,0,0,0,0,0,0,1,1,0
2,-0.290592,-1.597759,-0.024017,-0.610609,-0.341792,-0.005702,-0.096597,1.735152,-0.218076,-0.002318,...,0,0,1,0,0,0,0,0,1,0
3,-1.508153,0.625877,-0.116302,1.217044,2.398067,-0.037864,-0.102282,1.694030,-0.217999,-0.002318,...,0,0,1,0,0,0,0,1,0,0
4,-1.508153,-1.597759,-0.116302,-0.558883,0.571495,-0.045904,-0.102282,1.750119,-0.217999,-0.002318,...,0,0,0,0,0,1,0,0,0,1


In [98]:
X_vif = train.drop(['Default_Status'],axis=1) 

In [121]:
Q1 = train_num.quantile(0.25)
Q3 = train_num.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

df_new = train_num[~((train_num< (Q1 - 1.5 * IQR)) |(train_num> (Q3 + 1.5 * IQR))).any(axis=1)]
df_new.shape

Business_Industry_Type_Code       326620.00
New Business                           1.00
Employees                             14.00
Gross Disbursed Amount          13785502.50
Term_years                             5.00
Jobs Retained                          4.00
Expected Company Income          6700000.00
Funds available with company     1078786.75
Gross_Apprv_Amount              13440000.00
Company Branch Code                    0.00
Jobs Generated                         1.00
Balance Left                           0.00
Amount Defaulted                       0.00
Final_Appved_Amount             10762500.00
dtype: float64


(61866, 14)

In [122]:
X = pd.concat([df_new,train_cat],axis=1)
X.shape

(195118, 66)